In [3]:
#!/usr/bin/env python
# coding: utf-8

# Import necessary libraries
import pandas as pd
import numpy as np
import joblib
import sqlite3

# Load the test data
test_data = pd.read_csv("../data/test.csv")

# Select 10 devices for prediction
test_samples = test_data.sample(10)
test_samples.to_csv("../data/test_samples.csv", index=False)  # Save selected samples

# Load the best logistic regression model
best_logistic_model = joblib.load('../models/best_logistic_regression_model.pkl')

# Load the scaler used during training
scaler = joblib.load('../models/scaler.pkl')

# Prepare the data as per the model requirements
# Ensure the columns are passed with the correct feature names
test_samples_prepared = pd.DataFrame(scaler.transform(test_samples.drop(columns=['id'])), columns=test_samples.drop(columns=['id']).columns)

# Predict the price range for the selected devices
predictions = best_logistic_model.predict(test_samples_prepared)

# Print the predictions
print("Predictions for 10 test devices:")
for i, prediction in enumerate(predictions):
    print(f"Device {i+1} (ID: {test_samples.iloc[i]['id']}): Predicted price range = {prediction}")

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('../data/devices_predictions.db')
cursor = conn.cursor()

# Create a table for storing predictions if it does not exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS predictions (
    id INTEGER PRIMARY KEY,
    device_id INTEGER,
    prediction INTEGER
)
''')

# Insert predictions into the database
for i, prediction in enumerate(predictions):
    cursor.execute('''
    INSERT INTO predictions (device_id, prediction)
    VALUES (?, ?)
    ''', (int(test_samples.iloc[i]['id']), int(prediction)))

# Commit the transaction and close the connection
conn.commit()
conn.close()

# Print confirmation message
print("Predictions have been stored in the database.")



Predictions for 10 test devices:
Device 1 (ID: 96.0): Predicted price range = 2
Device 2 (ID: 136.0): Predicted price range = 1
Device 3 (ID: 239.0): Predicted price range = 3
Device 4 (ID: 330.0): Predicted price range = 0
Device 5 (ID: 389.0): Predicted price range = 3
Device 6 (ID: 496.0): Predicted price range = 0
Device 7 (ID: 598.0): Predicted price range = 3
Device 8 (ID: 328.0): Predicted price range = 3
Device 9 (ID: 672.0): Predicted price range = 1
Device 10 (ID: 690.0): Predicted price range = 2
Predictions have been stored in the database.
